In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
import lxml

Now that necessary packages are installed, going to parse the needed url. This URL has stats for each player in the league for the 2016-17 season, as well as contract information only for the year of 2016-17. So I am going to scrape the cap friendly website (Players Browse tab) and get this into a dataframe that I can download and put into the MySQL database.

In [2]:
url_v3 = "https://www.capfriendly.com/browse/active/2017?stats-season=2017&display=signing-team,birthday,country,weight,height,weightkg,heightcm,draft,slide-candidate,waivers-exempt,signing-status,expiry-year,performance-bonus,signing-bonus,caphit-percent,aav,length,minors-salary,base-salary,skater-individual-advanced-stats,skater-on-ice-advanced-stats,goalie-advanced-stats,arbitration-eligible,type,signing-age,signing-date,arbitration,extension&limits=gp-5-90"

req2 = requests.get(url_v3)
soup = BeautifulSoup(req2.content)  # make a soup of html & css from the web page

In [3]:
df = pd.read_html(url_v3, header=0, index_col = 0, na_values=["-"])[0]

In [4]:
df.shape

(50, 69)

After running df.shape, we can see our dataframe has 50 rows (players) and 69 columns (attributes about that player for the 2023-24 season). There are more than 50 players but they are on different url links technically as the table on that specific url only shows 50 players. So we need to retrieve the rest of the players. 

Scraping multiple pages of the main table

In [5]:
info_about_lists = soup.find_all("a", {"class": "whi pagin_r"})  # via devtools we find the element that allows to switch between pages of data
     

In [6]:
print(info_about_lists)  # all links to other pages of data

[<a class="whi pagin_r" data-val="2" href="/browse/?p=2">2</a>, <a class="whi pagin_r" data-val="3" href="/browse/?p=3">3</a>, <a class="whi pagin_r" data-val="18" href="/browse/?p=18">Last</a>]


In [7]:
last_list_num = int(info_about_lists[-1]["data-val"])  # take the last number of page from date-val so we now how many values were selected for us
     

In [8]:
print(last_list_num)  # check that 18th is last page number we got

18


Now we can use a for loop to parse all the data we have on multiple pages

In [9]:
req = requests.get(url_v3)
soup = BeautifulSoup(req.content)  # make a soup of html & css from the web page

info_about_lists = soup.find_all("a", {"class": "whi pagin_r"})  # via devtools we find the element that allows to switch between pages of data
last_list_num = int(info_about_lists[-1]["data-val"])  # take the last number of page from date-val so we now how many values were selected for us

pages_dfs = []

url_start = "https://www.capfriendly.com/browse/active/2017?stats-season=2017&display=signing-team,birthday,country,weight,height,weightkg,heightcm,draft,slide-candidate,waivers-exempt,signing-status,expiry-year,performance-bonus,signing-bonus,caphit-percent,aav,length,minors-salary,base-salary,skater-individual-advanced-stats,skater-on-ice-advanced-stats,goalie-advanced-stats,arbitration-eligible,type,signing-age,signing-date,arbitration,extension&limits=gp-5-90"

for page_num in range(1, last_list_num + 1):

        print(f"Start scapring page {page_num}")

        time.sleep(1)  # let the page download the results

        url = url_start + f"&pg={page_num}"  # we parse the needed page by adding a parameter for url
        df = pd.read_html(url, header=0, index_col = 0, na_values=["-"])[0]

        df = df.reset_index()  # to have player name as a separate column

        print(df.shape[0], f"rows were retrieved from page number {page_num}")

        pages_dfs.append(df)


result_df = pd.concat(pages_dfs)

Start scapring page 1
50 rows were retrieved from page number 1
Start scapring page 2
50 rows were retrieved from page number 2
Start scapring page 3
50 rows were retrieved from page number 3
Start scapring page 4
50 rows were retrieved from page number 4
Start scapring page 5
50 rows were retrieved from page number 5
Start scapring page 6
50 rows were retrieved from page number 6
Start scapring page 7
50 rows were retrieved from page number 7
Start scapring page 8
50 rows were retrieved from page number 8
Start scapring page 9
50 rows were retrieved from page number 9
Start scapring page 10
50 rows were retrieved from page number 10
Start scapring page 11
50 rows were retrieved from page number 11
Start scapring page 12
50 rows were retrieved from page number 12
Start scapring page 13
50 rows were retrieved from page number 13
Start scapring page 14
50 rows were retrieved from page number 14
Start scapring page 15
50 rows were retrieved from page number 15
Start scapring page 16
50 ro

In [10]:
result_df.head(5)

,PLAYER,TEAM,AGE,DATE OF BIRTH,COUNTRY,WEIGHT,HEIGHT,POS,HANDED,DRAFTED,...,EXPIRY,EXP. YEAR,CAP HIT,CAP HIT %,AAV,SALARY,BASE SALARY,MINORS,S.BONUS,P.BONUS
0,1. Jonathan Toews,NaN,28,"Apr. 29, 1988",Canada,201 lbs - 91 kg,"6'2"" - 188 cm",C,Left,3 - Round 1 - 2006 (CHI),...,UFA,2023,"$10,500,000",15.2%,"$10,500,000","$13,800,000","$7,800,000","$13,800,000","$6,000,000",$0
1,2. Patrick Kane,DET,27,"Nov. 19, 1988",United States,177 lbs - 80 kg,"5'10"" - 178 cm",RW,Left,1 - Round 1 - 2007 (CHI),...,UFA,2023,"$10,500,000",15.2%,"$10,500,000","$13,800,000","$7,800,000","$13,800,000","$6,000,000",$0
2,3. Anze Kopitar,LAK,28,"Aug. 24, 1987",Slovenia,225 lbs - 102 kg,"6'3"" - 191 cm",C,Left,11 - Round 1 - 2005 (LAK),...,UFA,2024,"$10,000,000",14.0%,"$10,000,000","$14,000,000","$5,000,000","$14,000,000","$9,000,000",$0
3,4. Alex Ovechkin,WSH,30,"Sep. 17, 1985",Russia,236 lbs - 107 kg,"6'3"" - 191 cm",LW,Right,1 - Round 1 - 2004 (WSH),...,UFA,2021,"$9,538,462",19.0%,"$9,538,462","$10,000,000","$10,000,000","$10,000,000",$0,$0
4,5. Evgeni Malkin,PIT,29,"Jul. 31, 1986",Russia,195 lbs - 88 kg,"6'3"" - 191 cm",C,Left,2 - Round 1 - 2004 (PIT),...,UFA,2022,"$9,500,000",14.8%,"$9,500,000","$9,500,000","$9,500,000","$9,500,000",$0,$0


Now, I have player statistics and cap info for the 2016-17 season. I have every player in the entire league for the 2016-17 season!!! So time to export it to a csv, and then upload it into the MySQL database I created. 

In [11]:
result_df.to_csv('Cap Friendly 2016-17 Player Data2.csv', encoding='utf-8')
result_df.to_csv('Cap Friendly 2016-17 Player Data.csv')

The csv file above is a weird file type **Forgot to add the .csv extension above, issue is fixed now. Just have to replace the weird "âœ” with a Yes as when a check mark is scraped off their website, it can not get represented correctly in excel. So just replace that symbol with the text Yes.

Now I have the 2016-17 CapFriendly Player Data all downloaded, time to download the 10 prior seasons(Include that seasons stats and contract information) and 8 seasons after (will include contract information). This will be done in a separate file to ensure this notebook does not get negatively impacted. 